### 📘 문서 QA용 메타데이터 자동 태깅 튜토리얼
이 노트북에서는 `LangChain`의 `create_metadata_tagger`를 사용하여 문서에 자동으로 메타데이터를 태깅하는 방법을 배웁니다.

이 방식은 RAG 시스템의 문서 검색 품질을 높이는 데 유용합니다. 특히 Late Chunking을 사용하는 경우, 문서 전체 맥락에 대한 메타 정보가 더 풍부할수록 검색 정밀도가 높아집니다.

#### 1️⃣ 메타데이터 스키마 정의
우리는 문서 제목, 어조, 길이 분류를 추출하는 스키마를 설계합니다. 이 정보는 나중에 벡터 검색 시 필터링/조건 검색에 사용됩니다.

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_community.document_transformers.openai_functions import create_metadata_tagger
from langchain_core.documents import Document
from typing import List
from pydantic import BaseModel, Field

class DocumentMetadata(BaseModel):
    country: List[str] = Field(
        description="문서에 언급된 국가들의 한글 표준화된 명칭 (예: '미국', '대한민국', '일본')"
    )
    organization: List[str] = Field(
        description="문서에 언급된 조직이나 기관의 이름을 원문 표기 그대로 추출"
    )
    policy: List[str] = Field(
        description="문서에 언급된 정책이나 제도의 이름을 원문 표기 그대로 추출"
    )
    year: List[int] = Field(
        description="문서 내용과 관련된 모든 연도 (문서의 작성 연도 및 문서에서 언급하는 모든 중요한 연도)"
    )

llm = init_chat_model("openai:gpt-4.1-mini", temperature=0)

# OpenAI Function 지원 모델 사용
document_transformer = create_metadata_tagger(DocumentMetadata, llm)

#### 2️⃣ 예시 문서에 태깅 수행
RAG 시스템에서 사용할 문서 예시를 구성하고 자동 태깅을 적용해봅니다.

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader("./data/국가별 공공부문 AI 도입 및 활용 전략.pdf")
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splitted_documents = splitter.split_documents(documents)

test_documents=splitted_documents[10:15]

enhanced_documents = document_transformer.transform_documents(test_documents)

In [ ]:
import json
for doc in enhanced_documents:
    print(doc.page_content[:100])
    print("\n📌 Metadata:", json.dumps(doc.metadata, ensure_ascii=False))
    print("\n" + "-" * 50 + "\n")

#### 🏷️ 메타데이터 태깅 비동기 함수 구현하기

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import nest_asyncio
import asyncio
import time
from concurrent.futures import ThreadPoolExecutor

nest_asyncio.apply()

# 비동기 래퍼 함수 구현
async def async_transform_document(transformer, doc):
    """단일 문서에 대한 비동기 변환 래퍼"""
    loop = asyncio.get_event_loop()
    with ThreadPoolExecutor() as executor:
        return await loop.run_in_executor(
            executor, 
            lambda: transformer.transform_documents([doc])[0]
        )

async def process_documents_async(transformer, documents, batch_size=5):
    """문서 배치를 비동기적으로 처리"""
    results = []
    
    # 문서를 배치로 나누기
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i+batch_size]
        
        # 각 배치에 대한 태스크 생성
        tasks = [async_transform_document(transformer, doc) for doc in batch]
        
        # 모든 태스크 동시 실행
        batch_results = await asyncio.gather(*tasks)
        results.extend(batch_results)
    
    return results

In [ ]:
async_results = await process_documents_async(document_transformer, test_documents, batch_size=5)

In [ ]:
async_results